# **作業四、 SRCNN在影像辨識與生成的練習**

### 姓名：謝元皓
### 學號：410978002
<hr>

### 資料介紹
- 91張花朵之清晰圖像及來自Set5、Set14資料集的19張清晰圖像。

- 32*32的兩萬張小圖。

- 製作兩萬張小圖的殘差圖，即原小圖減掉模糊小圖的圖片。
### 作品目標
這份作品基於 SRCNN (Super-Resolution Convolutional Neural Network) 模型，進行高解析度影像生成的訓練和測試。與上課下載的程式碼不同之處包括：

1. 額外的訓練次數：在原本的預訓練基礎上進行了額外的兩次訓練。
2. PSNR 計算和展示：在每個訓練週期後計算並展示 Train PSNR 和 Val PSNR。
3. 測試與展示：對 Set5 和 Set14 測試集進行測試，並展示其 Test PSNR。
4. 影像放大功能：編寫程式對已訓練完成的模型，輸入一張影像並進行放大處理，並與 Bicubic 結果進行對比。

<HR>

### 一、讀入 pre-trained 的 pth 檔，再進行 2 次 training，並展示其 Train PSNR 與 Val PSNR 值


In [8]:
dirThis = 'D:\python_venv\SRCNN' # 檔案目錄

import sys # 添加路徑
sys.path.append(dirThis + 'src/')
import os # 創資料夾
import time # 計時
import SRCNN_srcnn_4 as srcnn # 我們的模型py檔
import numpy as np # 使用ndarray及相關計算
import pandas as pd # 讀資料
from PIL import Image # 處理圖片
from tqdm import tqdm # 進度條
import matplotlib.pyplot as plt # 畫圖
from skimage.color import rgb2ycbcr, ycbcr2rgb # YCbCr操作
# from datasets import get_datasets, get_dataloaders # 抓取資料集
# from utils import (
#     psnr, save_model, save_model_state, 
#     save_plot, save_validation_results
# ) # 訓練相關

# torch系列
import torch
import torch.optim as optim
import torch.nn as nn

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Howard\AppData\Local\Temp\ipykernel_29908\2704681886.py:1: SyntaxWarning: invalid escape sequence '\p'
  dirThis = 'D:\python_venv\SRCNN' # 檔案目錄


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage import io, transform
import glob
import sys # 添加路徑
sys.path.append(dirThis + 'input/Set5')

# 定義數據集
class CustomDataset(Dataset):
    def __init__(self, image_dir, scale_factor):
        self.image_files = glob.glob(os.path.join(image_dir, '*.png'))
        self.scale_factor = scale_factor

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img = io.imread(self.image_files[idx], as_gray=True)
        img_lr = transform.rescale(img, 1.0 / self.scale_factor, anti_aliasing=True, multichannel=False)
        img_lr = transform.rescale(img_lr, self.scale_factor, anti_aliasing=False, multichannel=False)
        img = torch.from_numpy(img).float().unsqueeze(0)
        img_lr = torch.from_numpy(img_lr).float().unsqueeze(0)
        return img_lr, img

# 訓練函數
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs=2):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_psnr = 0
        for data in train_loader:
            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_psnr += psnr(targets.cpu().numpy(), outputs.cpu().detach().numpy())

        model.eval()
        val_psnr = 0
        with torch.no_grad():
            for data in val_loader:
                inputs, targets = data
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                val_psnr += psnr(targets.cpu().numpy(), outputs.cpu().detach().numpy())

        print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}, Train PSNR: {train_psnr/len(train_loader)}, Val PSNR: {val_psnr/len(val_loader)}')

# 設定裝置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 初始化模型、損失函數和優化器
model = srcnn().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 加載數據集
train_dataset = CustomDataset('train_images', scale_factor=3)
val_dataset = CustomDataset('val_images', scale_factor=3)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# 訓練模型並保存預訓練權重
train(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)
torch.save(model.state_dict(), 'srcnn_pretrained.pth')

# 加載預訓練模型並進行額外訓練
model.load_state_dict(torch.load('srcnn_pretrained.pth'))
train(model, criterion, optimizer, train_loader, val_loader, num_epochs=2)

# 保存模型
torch.save(model.state_dict(), 'srcnn_finetuned.pth')


TypeError: 'module' object is not callable